<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [27]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [28]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [29]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [32]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(response.text, "html.parser")
print(soup.title)  # Para verificar que el HTML se ha obtenido correctamente

url_spacex = "https://api.spacexdata.com/v4/launches"
response = requests.get(url_spacex)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    data = response.json()  # Aquí sí puedes usar `.json()`
    df = pd.json_normalize(data)  # Convertir en DataFrame
    print(df.head())  # Mostrar las primeras filas
else:
    print(f"Error al obtener datos: {response.status_code}")






<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>
       static_fire_date_utc  static_fire_date_unix    net  window  \
0  2006-03-17T00:00:00.000Z           1.142554e+09  False     0.0   
1                      None                    NaN  False     0.0   
2                      None                    NaN  False     0.0   
3  2008-09-20T00:00:00.000Z           1.221869e+09  False     0.0   
4                      None                    NaN  False     0.0   

                     rocket success  \
0  5e9d0d95eda69955f709d1eb   False   
1  5e9d0d95eda69955f709d1eb   False   
2  5e9d0d95eda69955f709d1eb   False   
3  5e9d0d95eda69955f709d1eb    True   
4  5e9d0d95eda69955f709d1eb    True   

                                            failures  \
0  [{'time': 33, 'altitude': None, 'reason': 'mer...   
1  [{'time': 301, 'altitude': 289, 'reason': 'har...   
2  [{'time': 140, 'altitude': 35, 'reason': 'resi...   
3                                                 []   


Create a `BeautifulSoup` object from the HTML `response`


In [37]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
from bs4 import BeautifulSoup

# Crear el objeto BeautifulSoup desde el contenido HTML de la respuesta
soup = BeautifulSoup(response.text, "html.parser")
print(df.columns)

Index(['static_fire_date_utc', 'static_fire_date_unix', 'net', 'window',
       'rocket', 'success', 'failures', 'details', 'crew', 'ships', 'capsules',
       'payloads', 'launchpad', 'flight_number', 'name', 'date_utc',
       'date_unix', 'date_local', 'date_precision', 'upcoming', 'cores',
       'auto_update', 'tbd', 'launch_library_id', 'id', 'fairings.reused',
       'fairings.recovery_attempt', 'fairings.recovered', 'fairings.ships',
       'links.patch.small', 'links.patch.large', 'links.reddit.campaign',
       'links.reddit.launch', 'links.reddit.media', 'links.reddit.recovery',
       'links.flickr.small', 'links.flickr.original', 'links.presskit',
       'links.webcast', 'links.youtube_id', 'links.article', 'links.wikipedia',
       'fairings'],
      dtype='object')


Print the page title to verify if the `BeautifulSoup` object was created properly 


In [40]:
# Use soup.title attribute
#print(soup.title.text)  # Muestra el título de la página
# Filtrar los lanzamientos que sean del Falcon 9
# Obtener datos sobre los cohetes
url_rockets = "https://api.spacexdata.com/v4/rockets"
response_rockets = requests.get(url_rockets)

# Verificar si la solicitud fue exitosa
if response_rockets.status_code == 200:
    rockets_data = response_rockets.json()
    
    # Crear un diccionario con los IDs y nombres de los cohetes
    rocket_names = {rocket['id']: rocket['name'] for rocket in rockets_data}
    
    # Agregar la columna de nombre real en el DataFrame
    df['rocket_name'] = df['rocket'].map(rocket_names)

    # Filtrar los lanzamientos de Falcon 9
    df_falcon9 = df[df['rocket_name'] == 'Falcon 9']
    print(f"Cantidad de lanzamientos de Falcon 9: {len(df_falcon9)}")
else:
    print(f"Error al obtener datos de cohetes: {response_rockets.status_code}")




Cantidad de lanzamientos de Falcon 9: 195


KeyError: 'landpad'

### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [8]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`
# Buscar todas las tablas en la página
html_tables = soup.find_all('table')

# Verificar cuántas tablas se encontraron
print(f"Se encontraron {len(html_tables)} tablas en la página.")


Se encontraron 25 tablas en la página.


Starting from the third table is our target table contains the actual launch records.


In [9]:
# Let's print the third table and check its content
first_launch_table = html_tables[2]
print(first_launch_table)

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11"><span class="cite-bracket">[</span>b<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12"><span class="cite-bracket">[</span>c<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [10]:
column_names = []

# Apply find_all() function with `th` element on first_launch_table
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names
# Iterar sobre los elementos <th> y extraer los nombres de columna
for th in first_launch_table.find_all('th'):
    name = extract_column_from_header(th)  # Aplicar la función
    
    # Agregar el nombre solo si no es vacío
    if name is not None and len(name) > 0:
        column_names.append(name)

Check the extracted column names


In [54]:
missing_landing_pad = df_cores['landpad'].isnull().sum()
print(f"Cantidad de valores faltantes en 'landpad': {missing_landing_pad}")
print(f"Total de valores en 'landpad': {df_cores['landpad'].count()}")
missing_landpad = df_cores['landpad'].isna().sum()
empty_landpad = (df_cores['landpad'] == "").sum()  # Cadenas vacías
null_landpad = df_cores['landpad'].isnull().sum()  # Valores nulos

print(f"Valores NaN en 'landpad': {missing_landpad}")
print(f"Valores vacíos en 'landpad': {empty_landpad}")
print(f"Valores nulos en 'landpad': {null_landpad}")
print(f"Total de valores faltantes en 'landpad': {missing_landpad + empty_landpad + null_landpad}")


'''df_falcon9 = df[(df['rocket_name'] == 'Falcon 9') & (df['success'] == True)]
print(f"Lanzamientos exitosos de Falcon 9: {len(df_falcon9)}")
print(df['success'].value_counts())
df_falcon9 = df[(df['rocket_name'] == 'Falcon 9') & (df['success'].notna())]
print(f"Lanzamientos de Falcon 9 que tienen un valor en 'success': {len(df_falcon9)}")
df_falcon9_total = df[df['rocket_name'] == 'Falcon 9']
print(f"Total de lanzamientos de Falcon 9 (sin importar éxito/falla): {len(df_falcon9_total)}")
df_falcon9_completados = df[(df['rocket_name'] == 'Falcon 9') & (df['success'] == True) & (df['upcoming'] == False)]
print(f"Lanzamientos completados y exitosos de Falcon 9: {len(df_falcon9_completados)}")
df_falcon9_completados = df[(df['rocket_name'] == 'Falcon 9') & (df['upcoming'] == False)]
print(f"Lanzamientos completados de Falcon 9: {len(df_falcon9_completados)}")
print(df['rocket_name'].value_counts())
df_falcon9_filtrado = df[(df['rocket_name'] == 'Falcon 9') & (df['rocket_name'] != 'Falcon 1')]
print(f"Lanzamientos de Falcon 9 después de eliminar Falcon 1: {len(df_falcon9_filtrado)}")
df_falcon9_filtrado = df[(df['rocket_name'] == 'Falcon 9') & (df['upcoming'] == False)]
print(f"Lanzamientos de Falcon 9 completados (sin Falcon 1): {len(df_falcon9_filtrado)}")'''






Cantidad de valores faltantes en 'landpad': 58
Total de valores en 'landpad': 157
Valores NaN en 'landpad': 58
Valores vacíos en 'landpad': 0
Valores nulos en 'landpad': 58
Total de valores faltantes en 'landpad': 116


'df_falcon9 = df[(df[\'rocket_name\'] == \'Falcon 9\') & (df[\'success\'] == True)]\nprint(f"Lanzamientos exitosos de Falcon 9: {len(df_falcon9)}")\nprint(df[\'success\'].value_counts())\ndf_falcon9 = df[(df[\'rocket_name\'] == \'Falcon 9\') & (df[\'success\'].notna())]\nprint(f"Lanzamientos de Falcon 9 que tienen un valor en \'success\': {len(df_falcon9)}")\ndf_falcon9_total = df[df[\'rocket_name\'] == \'Falcon 9\']\nprint(f"Total de lanzamientos de Falcon 9 (sin importar éxito/falla): {len(df_falcon9_total)}")\ndf_falcon9_completados = df[(df[\'rocket_name\'] == \'Falcon 9\') & (df[\'success\'] == True) & (df[\'upcoming\'] == False)]\nprint(f"Lanzamientos completados y exitosos de Falcon 9: {len(df_falcon9_completados)}")\ndf_falcon9_completados = df[(df[\'rocket_name\'] == \'Falcon 9\') & (df[\'upcoming\'] == False)]\nprint(f"Lanzamientos completados de Falcon 9: {len(df_falcon9_completados)}")\nprint(df[\'rocket_name\'].value_counts())\ndf_falcon9_filtrado = df[(df[\'rocket_name\']

## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [12]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [13]:
extracted_row = 0
#Extract each table 
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row 
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number 
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #get table element 
        row=rows.find_all('td')
        #if it is number save cells in a dictonary 
        if flag:
            extracted_row += 1
            # Flight Number value
            # TODO: Append the flight_number into launch_dict with key `Flight No.`
            #print(flight_number)
            datatimelist=date_time(row[0])
            
            # Date value
            # TODO: Append the date into launch_dict with key `Date`
            date = datatimelist[0].strip(',')
            #print(date)
            
            # Time value
            # TODO: Append the time into launch_dict with key `Time`
            time = datatimelist[1]
            #print(time)
              
            # Booster version
            # TODO: Append the bv into launch_dict with key `Version Booster`
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            print(bv)
            
            # Launch Site
            # TODO: Append the bv into launch_dict with key `Launch Site`
            launch_site = row[2].a.string
            #print(launch_site)
            
            # Payload
            # TODO: Append the payload into launch_dict with key `Payload`
            payload = row[3].a.string
            #print(payload)
            
            # Payload Mass
            # TODO: Append the payload_mass into launch_dict with key `Payload mass`
            payload_mass = get_mass(row[4])
            #print(payload)
            
            # Orbit
            # TODO: Append the orbit into launch_dict with key `Orbit`
            orbit = row[5].a.string
            #print(orbit)
            
            # Customer
            # TODO: Append the customer into launch_dict with key `Customer`
            customer = row[6].a.string
            #print(customer)
            
            # Launch outcome
            # TODO: Append the launch_outcome into launch_dict with key `Launch outcome`
            launch_outcome = list(row[7].strings)[0]
            #print(launch_outcome)
            
            # Booster landing
            # TODO: Append the launch_outcome into launch_dict with key `Booster landing`
            booster_landing = landing_status(row[8])
            #print(booster_landing)
            

F9 v1.07B0003.18
F9 v1.07B0004.18
F9 v1.07B0005.18
F9 v1.07B0006.18
F9 v1.07B0007.18
F9 v1.17B10038
F9 v1.1
F9 v1.1
F9 v1.1
F9 v1.1
F9 v1.1
F9 v1.1[
F9 v1.1[
F9 v1.1[
F9 v1.1[
F9 v1.1[
F9 v1.1[
F9 v1.1[
F9 v1.1[
F9 FT[
F9 v1.1[
F9 FT[
F9 FT[
F9 FT[
F9 FT[
F9 FT[
F9 FT[
F9 FT[
F9 FT[
F9 FT[
F9 FT[
F9 FT♺[
F9 FT[
F9 FT[
F9 FT[
F9 FTB1029.2195
F9 FT[
F9 FT[
F9 B4[
F9 FT[
F9 B4[
F9 B4[
F9 FTB1031.2220
F9 B4[
F9 FTB1035.2227
F9 FTB1036.2227
F9 B4[
F9 FTB1032.2245
F9 FTB1038.2268
F9 B4[
F9 B4B1041.2268
F9 B4B1039.2292
F9 B4[
F9 B5311B1046.1268
F9 B4B1043.2322
F9 B4B1040.2268
F9 B4B1045.2336
F9 B5
F9 B5349B1048[
F9 B5B1046.2354
F9 B5[
F9 B5B1048.2364
F9 B5B1047.2268
F9 B5B1046.3268
F9 B5[
F9 B5[
F9 B5B1049.2397
F9 B5B1048.3399
F9 B5[]413
F9 B5[
F9 B5B1049.3434
F9 B5B1051.2420
F9 B5B1056.2465
F9 B5B1047.3472
F9 B5
F9 B5[
F9 B5B1056.3482
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5[
F9 B5
F9 B5
F9 B5
F9 B5B1058.2544
F9 B5
F9 B5B1049.6544
F9 B5
F9 B5B1060.2563
F9 B5B1058.3565
F9 B5B1051.6568


AttributeError: 'NoneType' object has no attribute 'string'

After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [21]:
  # Muestra la estructura completa de la tabla
print(tables[2].prettify())  # Muestra la estructura completa de la tabla
for row in tables[2].find_all("tr")[:5]:  # Verifica las primeras 5 filas
    print(row.prettify())  # Muestra la estructura de cada fila
for rows in tables[2].find_all("tr"):
    row = rows.find_all("td")

    if row:  # Verificar si la fila tiene datos
        print([cell.text.strip() for cell in row])  # Muestra los valores extraídos de cada celda
for key, value in launch_dict.items():
    print(f"{key}: {value[:5]}")  # Muestra los primeros 5 elementos de cada lista
launch_dict['Flight No.'].append(flight_number)
print(f"Agregando: {flight_number}, {date}, {time}, {payload}")
df = pd.DataFrame({key: pd.Series(value, dtype="object") for key, value in launch_dict.items()})
df.to_csv('spacex_web_scraped.csv', index=False)
print("Archivo 'spacex_web_scraped.csv' creado exitosamente.")





<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
 <tbody>
  <tr>
   <th scope="col">
    Flight No.
   </th>
   <th scope="col">
    Date and
    <br/>
    time (
    <a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">
     UTC
    </a>
    )
   </th>
   <th scope="col">
    <a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">
     Version,
     <br/>
     Booster
    </a>
    <sup class="reference" id="cite_ref-booster_11-2">
     <a href="#cite_note-booster-11">
      <span class="cite-bracket">
       [
      </span>
      b
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </th>
   <th scope="col">
    Launch site
   </th>
   <th scope="col">
    Payload
    <sup class="reference" id="cite_ref-Dragon_12-2">
     <a href="#cite_note-Dragon-12">
      <span class="cite-bracket">
       [
      </span>
      c
      <span class="cite-bracket">
       ]
  

We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
